# Blue Brain BioExplorer

![](../bioexplorer_banner.png)

## Visualization of a Coronavirus

In [ ]:
from bioexplorer import BioExplorer, Protein, Membrane, Cell, Sugar, \
                        AnimationParams, Vector2, Vector3, Quaternion, MovieMaker
be = BioExplorer('localhost:5000')
core = be.core_api()
be.reset_scene()
print(be.version())

## Resources

In [ ]:
resource_folder = '../../tests/test_files/'
pdb_folder = resource_folder + 'pdb/'

glycan_folder = pdb_folder + 'glycans/'
complex_paths = [glycan_folder + 'complex/5.pdb', glycan_folder + 'complex/15.pdb',
                 glycan_folder + 'complex/25.pdb',glycan_folder + 'complex/35.pdb']
high_mannose_paths = [glycan_folder + 'high-mannose/1.pdb', 
                      glycan_folder + 'high-mannose/2.pdb',
                      glycan_folder + 'high-mannose/3.pdb',
                      glycan_folder + 'high-mannose/4.pdb']
hybrid_paths = [glycan_folder + 'hybrid/20.pdb']
o_glycan_paths = [glycan_folder + 'o-glycan/1.pdb']

In [ ]:
status = be.core_api().set_camera(
    orientation=[-0.005, 0.439, 0.0, 0.898],
    position=[265.706, -34.254, 119.18],
    target=[71.9, -36.859, -31.474]
)

In [ ]:
status = be.set_rendering_quality(be.RENDERING_QUALITY_HIGH)

In [ ]:
# Scene
scene_size = Vector3(500.0, 500.0, 500.0)

# Proteins
protein_radius_multiplier = 1.0
protein_representation = be.REPRESENTATION_ATOMS

In [ ]:
def add_cell(nb_receptors, random_seed, random_position_seed, random_rotation_seed):
    name='Cell'

    '''ACE2 receptor definition'''
    ace2_receptor = Protein(
        name=name + '_' + be.NAME_RECEPTOR,
        source=pdb_folder + '6m18.pdb',
        occurences=nb_receptors,
        transmembrane_params=Vector2(-6.0, 5.0))

    '''Membrane definition'''
    membrane_size = Vector3(scene_size.x, scene_size.y / 10.0, scene_size.z)
    membrane = Membrane(
        lipid_sources=[pdb_folder + 'membrane/popc.pdb'],
        animation_params=AnimationParams(
            random_seed, random_position_seed, 0.25, random_rotation_seed, 0.2)
        )

    '''Cell definition'''
    cell = Cell(
        name=name,
        shape=be.ASSEMBLY_SHAPE_SINUSOID,
        shape_params=membrane_size,
        membrane=membrane, proteins=[ace2_receptor]
    )

    '''Add cell to scene'''
    status = be.add_cell(
        cell=cell, representation=protein_representation,
        atom_radius_multiplier=protein_radius_multiplier,
        position=Vector3(0.0, -80.0, 0.0)
    )

    '''Glycans'''
    be.add_multiple_glycans(
        representation=protein_representation, assembly_name=name, 
        glycan_type=be.NAME_GLYCAN_COMPLEX,
        protein_name=be.NAME_RECEPTOR, paths=complex_paths, 
        indices=[53, 90, 103, 322, 432, 690])

    be.add_multiple_glycans(
        representation=protein_representation, assembly_name=name,
        glycan_type=be.NAME_GLYCAN_HYBRID,
        protein_name=be.NAME_RECEPTOR, paths=hybrid_paths, 
        indices=[546])

    indices = [[164, Quaternion(0.707, 0.0, 0.707, 0.0)],
               [739, Quaternion(0.707, 0.0, 0.707, 0.0)]]
    for index in indices:
        o_glycan_name = name + '_' + be.NAME_GLYCAN_O_GLYCAN + '_' + str(index[0])
        o_glycan = Sugar(
            assembly_name=name, name=o_glycan_name, 
            source=o_glycan_paths[0],
            protein_name=name + '_' + be.NAME_RECEPTOR, 
            representation=protein_representation,
            chain_ids=[2, 4], site_indices=[index[0]], 
            rotation=index[1])
        be.add_sugars(o_glycan)
    return status

In [ ]:
def add_virus(position, random_seed, random_position_seed, random_rotation_seed, morphing_step):
    be.add_coronavirus(
        name='Coronavirus', resource_folder=resource_folder,
        atom_radius_multiplier=protein_radius_multiplier,
        representation=protein_representation,
        add_glycans=True,
        position=position,
        animation_params=AnimationParams(
            random_seed, random_position_seed, 0.25, random_rotation_seed + 1, 0.2, morphing_step)
    )

In [ ]:
from IPython.display import clear_output

mm = MovieMaker(be)
output_folder = '/tmp'
be.set_general_settings(
    model_visibility_on_creation=False, off_folder='/tmp/')

nb_frames = 40
nb_going_down_frames = 20
going_down_distance = 33.0

for frame in range(25, nb_going_down_frames + nb_frames, 5):
# for frame in range(nb_going_down_frames + nb_frames):
    clear_output()
    core.set_application_parameters(image_stream_fps=0)
    print('Frame %d' % frame)
    
    '''Cell'''
    status = add_cell(5, 1, 5 * frame + 1, 5 * frame + 2)
    
    '''Virus'''
    y = -5.0 - float(frame) * (going_down_distance / float(nb_going_down_frames))
    s = 0.0
    if frame >= nb_going_down_frames:
        y = -5.0 - going_down_distance
        s = float(frame - nb_going_down_frames) / float(nb_frames)
        
    add_virus(
        Vector3(55.0, y, -45),
        5, 5 * frame + 3, 5 * frame + 4, s
    )

    '''Materials'''
    be.apply_default_color_scheme(
        shading_mode=be.SHADING_MODE_BASIC, user_parameter=1)
    be.set_models_visibility(True)
    
    '''Snapshot'''
    mm.create_snapshot(
        renderer='bio_explorer',
        size=[540, 540], samples_per_pixel=32,
        path=output_folder, base_name='%05d' % frame)
        
status = core.set_application_parameters(image_stream_fps=20)